In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 재현성을 위해 시드 설정
tf.random.set_seed(777)

# MinMaxScaler 함수 (정규화)
def MinMaxScaler(data):
    numerator = data - np.min(data, axis=0)
    denominator = np.max(data, axis=0) - np.min(data, axis=0)
    return numerator / (denominator + 1e-7)

# 하이퍼파라미터
seq_length = 7
data_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 500

In [6]:

# 데이터 로드 (Open, High, Low, Volume, Close)
xy = np.loadtxt('data-02-stock_daily.csv', delimiter=',',skiprows=1)
xy = xy[::-1]  # 시간 순서대로 정렬

# train/test 데이터 나누기
train_size = int(len(xy) * 0.7)
train_set = xy[:train_size]
test_set = xy[train_size - seq_length:]  # 과거 시퀀스를 고려해서 설정

# 정규화
train_set = MinMaxScaler(train_set)
test_set = MinMaxScaler(test_set)

# 데이터셋 구성 함수
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]  # 시퀀스
        _y = time_series[i + seq_length, [-1]]  # 다음 Close 가격
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)


In [12]:
# 1.model class define
class LSTMModel(tf.keras.Model):
    def __init__(self):
        super(LSTMModel,self).__init__()
        self.lstm=tf.keras.layers.LSTM(units =hidden_dim,return_sequences=False,input_shape=(seq_length, data_dim))
        self.fc=tf.keras.layers.Dense(units =output_dim)
        
    def call(self,x):
        x= self.lstm(x)
        x=self.fc(x)
        return x
    
    

In [13]:
#2. model create
model =LSTMModel()
# Loss and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

#3. model compile
model.compile(optimizer=optimizer,loss=loss_fn)

# 4. Model learning
model.fit(trainX, trainY, epochs=iterations)



Epoch 1/500


/Users/onseonghyeon/miniforge3/envs/newenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 1/16 ━━━━━━━━━━━━━━━━━━━━ 9s 621ms/step - loss: 0.0000e+00

2024-10-01 14:45:32.161587: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2024-10-01 14:45:32.161604: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
	 [[{{function_node __inference_one_step_on_data_2705}}{{node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/onseonghyeon/miniforge3/envs/newenv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/onseonghyeon/miniforge3/envs/newenv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/onseonghyeon/miniforge3/envs/newenv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/onseonghyeon/miniforge3/envs/newenv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Users/onseonghyeon/miniforge3/envs/newenv/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/Users/onseonghyeon/minifo

In [ ]:
# 5. 예측
predicted_stock_price = model.predict(testX)

# 6. RMSE 계산
rmse = np.sqrt(np.mean(np.square(testY - predicted_stock_price)))
print("RMSE: {}".format(rmse))

# 7. 예측 결과 그래프
plt.plot(testY, label='Actual Stock Price')
plt.plot(predicted_stock_price, label='Predicted Stock Price')
plt.xlabel("Time Period")
plt.ylabel("Stock Price")
plt.legend()
plt.show()